In [1]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Models
from sklearn.neighbors import NearestNeighbors

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("resources/games_clean.csv")
print(df.shape)
df.head()

(9507, 17)


,game_id,max_players,max_playtime,min_age,min_players,min_playtime,name,playing_time,category,mechanic,average_rating,users_rated,category_count,mechanic_count,has_expansion,len_description,description_sentiment
0,1,5,240,14,3,240,Die Macher,240,economic,area control / area influence,7.66508,4498,3,5,0,222,0.091012
1,2,4,30,12,3,30,Dragonmaster,30,card game,trick-taking,6.60815,478,2,1,0,154,0.055291
2,3,4,60,10,2,30,Samurai,60,abstract strategy,area control / area influence,7.44119,12019,2,4,0,183,-0.056250
3,4,4,60,12,2,60,Tal der Könige,60,ancient,action point allowance system,6.60675,314,1,4,0,104,-0.007908
4,5,6,90,12,3,90,Acquire,90,economic,hand management,7.35830,15195,1,3,0,191,0.168056


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9507 entries, 0 to 9506
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   game_id                9507 non-null   int64  
 1   max_players            9507 non-null   int64  
 2   max_playtime           9507 non-null   int64  
 3   min_age                9507 non-null   int64  
 4   min_players            9507 non-null   int64  
 5   min_playtime           9507 non-null   int64  
 6   name                   9507 non-null   object 
 7   playing_time           9507 non-null   int64  
 8   category               9507 non-null   object 
 9   mechanic               9507 non-null   object 
 10  average_rating         9507 non-null   float64
 11  users_rated            9507 non-null   int64  
 12  category_count         9507 non-null   int64  
 13  mechanic_count         9507 non-null   int64  
 14  has_expansion          9507 non-null   int64  
 15  len_

In [5]:
# Drop null song names/artists
df = df.dropna(how="any").reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9507 entries, 0 to 9506
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   game_id                9507 non-null   int64  
 1   max_players            9507 non-null   int64  
 2   max_playtime           9507 non-null   int64  
 3   min_age                9507 non-null   int64  
 4   min_players            9507 non-null   int64  
 5   min_playtime           9507 non-null   int64  
 6   name                   9507 non-null   object 
 7   playing_time           9507 non-null   int64  
 8   category               9507 non-null   object 
 9   mechanic               9507 non-null   object 
 10  average_rating         9507 non-null   float64
 11  users_rated            9507 non-null   int64  
 12  category_count         9507 non-null   int64  
 13  mechanic_count         9507 non-null   int64  
 14  has_expansion          9507 non-null   int64  
 15  len_

In [6]:
# Select string columns
string_cols = df.select_dtypes(include=['object'])

# Count unique categories for each string column
unique_counts = string_cols.nunique()
unique_counts

name        9372
category      82
mechanic      51
dtype: int64

In [7]:
df2=df.copy()

In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9507 entries, 0 to 9506
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   game_id                9507 non-null   int64  
 1   max_players            9507 non-null   int64  
 2   max_playtime           9507 non-null   int64  
 3   min_age                9507 non-null   int64  
 4   min_players            9507 non-null   int64  
 5   min_playtime           9507 non-null   int64  
 6   name                   9507 non-null   object 
 7   playing_time           9507 non-null   int64  
 8   category               9507 non-null   object 
 9   mechanic               9507 non-null   object 
 10  average_rating         9507 non-null   float64
 11  users_rated            9507 non-null   int64  
 12  category_count         9507 non-null   int64  
 13  mechanic_count         9507 non-null   int64  
 14  has_expansion          9507 non-null   int64  
 15  len_

In [9]:
df2.columns

Index(['game_id', 'max_players', 'max_playtime', 'min_age', 'min_players',
       'min_playtime', 'name', 'playing_time', 'category', 'mechanic',
       'average_rating', 'users_rated', 'category_count', 'mechanic_count',
       'has_expansion', 'len_description', 'description_sentiment'],
      dtype='object')

In [10]:
# set columns
meta_cols = ['game_id', 'name']
feature_cols = [ 'max_players', 'max_playtime', 'min_age', 'min_players',
       'min_playtime', 'playing_time', 'category', 'mechanic',
       'average_rating', 'users_rated', 'category_count', 'mechanic_count',
       'has_expansion', 'len_description', 'description_sentiment']

In [11]:
# Define Preprocessing Pipelines

# Define preprocessing for numeric features
numeric_features = [ 'max_players', 'max_playtime', 'min_age', 'min_players',
       'min_playtime', 'playing_time', 
       'average_rating', 'users_rated', 'category_count', 'mechanic_count',
       'len_description', 'description_sentiment'] # You can also do this in a loop, select the numeric columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

# Define preprocessing for the binary features
binary_features = ['has_expansion']
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),
    ('label', OrdinalEncoder())])  # Label encode for binary feature

# Define preprocessing for categorical features
categorical_features = ['category', 'mechanic']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing for numeric and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('binary', binary_transformer, binary_features),
        ('cat', categorical_transformer, categorical_features)])

In [13]:
# Correlation Analysis (this is optional and not needed for the ML Experiment)
# It just shows what the data looks like after transformation before training
# We will still declare a full pipeline of preprocessing + training

# Use only preprocessing pipeline to transform the data
preprocessed_X_train = preprocessor.fit_transform(df2).toarray()

# Convert preprocessed data to a DataFrame
# Get the feature names after one-hot encoding
encoded_feature_names = (numeric_features + binary_features +
                         list(preprocessor.transformers_[2][1]['onehot'].get_feature_names_out(categorical_features)))

df_final = pd.DataFrame(preprocessed_X_train, columns=encoded_feature_names)
df_final[meta_cols] = df2.loc[:, meta_cols]
df_final.head()

,max_players,max_playtime,min_age,min_players,min_playtime,playing_time,average_rating,users_rated,category_count,mechanic_count,len_description,description_sentiment,has_expansion,category_abstract strategy,category_action / dexterity,category_adventure,category_age of reason,category_american civil war,category_american indian wars,category_american revolutionary war,category_american west,category_ancient,category_animals,category_arabian,category_aviation / flight,category_bluffing,category_book,category_card game,category_children's game,category_city building,category_civil war,category_civilization,category_collectible components,category_comic book / strip,category_deduction,category_dice,category_economic,category_educational,category_electronic,category_environmental,category_expansion for base-game,category_exploration,category_fantasy,category_farming,category_fighting,category_game system,category_horror,category_humor,category_industry / manufacturing,category_korean war,category_mafia,category_math,category_mature / adult,category_maze,category_medical,category_medieval,category_memory,category_miniatures,category_modern warfare,category_movies / tv / radio theme,category_murder/mystery,category_music,category_mythology,category_napoleonic,category_nautical,category_negotiation,category_novel-based,category_party game,category_pike and shot,category_pirates,category_political,category_post-napoleonic,category_prehistoric,category_print & play,category_puzzle,category_racing,category_real-time,category_religious,category_renaissance,category_science fiction,category_space exploration,category_spies/secret agents,category_sports,category_territory building,category_trains,category_transportation,category_travel,category_trivia,category_video game theme,category_vietnam war,category_wargame,category_word game,category_world war i,category_world war ii,category_zombies,mechanic_acting,mechanic_action / movement programming,mechanic_action point allowance system,mechanic_area control / area influence,mechanic_area enclosure,mechanic_area movement,mechanic_area-impulse,mechanic_auction/bidding,mechanic_betting/wagering,mechanic_campaign / battle card driven,mechanic_card drafting,mechanic_chit-pull system,mechanic_co-operative play,mechanic_commodity speculation,mechanic_crayon rail system,mechanic_deck / pool building,mechanic_dice rolling,mechanic_grid movement,mechanic_hand management,mechanic_hex-and-counter,mechanic_line drawing,mechanic_memory,mechanic_modular board,mechanic_paper-and-pencil,mechanic_partnerships,mechanic_pattern building,mechanic_pattern recognition,mechanic_pick-up and deliver,mechanic_player elimination,mechanic_point to point movement,mechanic_press your luck,mechanic_rock-paper-scissors,mechanic_role playing,mechanic_roll / spin and move,mechanic_route/network building,mechanic_secret unit deployment,mechanic_set collection,mechanic_simulation,mechanic_simultaneous action selection,mechanic_singing,mechanic_stock holding,mechanic_storytelling,mechanic_take that,mechanic_tile placement,mechanic_time track,mechanic_trading,mechanic_trick-taking,mechanic_variable phase order,mechanic_variable player powers,mechanic_voting,mechanic_worker placement,game_id,name
0,-0.025793,0.206217,1.216348,1.407807,0.230367,0.206217,1.484124,1.178580,0.253625,1.682974,0.173831,0.315755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Die Macher
1,-0.086458,-0.096368,0.633186,1.407807,-0.082570,-0.096368,0.218884,-0.152598,-0.49426

In [14]:
corrs = df_final.loc[:, [x for x in df_final.columns if x not in meta_cols]].corr()
corrs

,max_players,max_playtime,min_age,min_players,min_playtime,playing_time,average_rating,users_rated,category_count,mechanic_count,len_description,description_sentiment,has_expansion,category_abstract strategy,category_action / dexterity,category_adventure,category_age of reason,category_american civil war,category_american indian wars,category_american revolutionary war,category_american west,category_ancient,category_animals,category_arabian,category_aviation / flight,category_bluffing,category_book,category_card game,category_children's game,category_city building,category_civil war,category_civilization,category_collectible components,category_comic book / strip,category_deduction,category_dice,category_economic,category_educational,category_electronic,category_environmental,category_expansion for base-game,category_exploration,category_fantasy,category_farming,category_fighting,category_game system,category_horror,category_humor,category_industry / manufacturing,category_korean war,category_mafia,category_math,category_mature / adult,category_maze,category_medical,category_medieval,category_memory,category_miniatures,category_modern warfare,category_movies / tv / radio theme,category_murder/mystery,category_music,category_mythology,category_napoleonic,category_nautical,category_negotiation,category_novel-based,category_party game,category_pike and shot,category_pirates,category_political,category_post-napoleonic,category_prehistoric,category_print & play,category_puzzle,category_racing,category_real-time,category_religious,category_renaissance,category_science fiction,category_space exploration,category_spies/secret agents,category_sports,category_territory building,category_trains,category_transportation,category_travel,category_trivia,category_video game theme,category_vietnam war,category_wargame,category_word game,category_world war i,category_world war ii,category_zombies,mechanic_acting,mechanic_action / movement programming,mechanic_action point allowance system,mechanic_area control / area influence,mechanic_area enclosure,mechanic_area movement,mechanic_area-impulse,mechanic_auction/bidding,mechanic_betting/wagering,mechanic_campaign / battle card driven,mechanic_card drafting,mechanic_chit-pull system,mechanic_co-operative play,mechanic_commodity speculation,mechanic_crayon rail system,mechanic_deck / pool building,mechanic_dice rolling,mechanic_grid movement,mechanic_hand management,mechanic_hex-and-counter,mechanic_line drawing,mechanic_memory,mechanic_modular board,mechanic_paper-and-pencil,mechanic_partnerships,mechanic_pattern building,mechanic_pattern recognition,mechanic_pick-up and deliver,mechanic_player elimination,mechanic_point to point movement,mechanic_press your luck,mechanic_rock-paper-scissors,mechanic_role playing,mechanic_roll / spin and move,mechanic_route/network building,mechanic_secret unit deployment,mechanic_set collection,mechanic_simulation,mechanic_simultaneous action selection,mechanic_singing,mechanic_stock holding,mechanic_storytelling,mechanic_take that,mechanic_tile placement,mechanic_time track,mechanic_trading,mechanic_trick-taking,mechanic_variable phase order,mechanic_variable player powers,mechanic_voting,mechanic_worker placement
max_players,1.000000,-0.002636,-0.011429,0.075155,-0.003320,-0.002636,-0.028721,-0.000860,0.023899,-0.011697,-0.016640,0.011805,-0.001116,-0.026651,0.029578,-0.008589,-0.010722,-0.020231,-0.004714,-0.003053,-0.003097,-0.012994,-0.003098,-0.002499,0.016463,0.026038,0.007503,0.040385,-0.002335,-0.007187,-0.007305,-0.003512,-0.009442,0.005584,0.004132,0.016524,-0.003059,-0.001382,0.004124,-0.002018,-0.002131,-0.003966,-0.008165,-0.001270,-0.005952,0.033314,-0.001957,0.018042,-0.002577,-0.004332,-0.000917,0.025571,0.006828,-0.000749,-0.002347,-0.008946,0.024137,-0.008510,-0.020344,-0.003363,0.000792,0.002112,-0.002412,-0.019580,-0.011254,-0.000618,-0.001816,0.044176,-0.008220,-0.002151,-0.002237,-0.008463,-0.000790,-0.005406,-0.004731,0.006472,0.000938,-

In [16]:
# plt.figure(figsize=(18,10))
# sns.heatmap(corrs, annot=True)
# plt.show()

In [17]:
# User inputs
playlist_length = 20
game = "Catan" #Circles


game_id = df2.loc[(df2.name == game) ].game_id.values[0]
game_id

13

In [18]:
# Step 1: Get the data (NOTE: this is the original dataframe, not the one used for the correlations)
X = df2.loc[:, feature_cols]

# Preprocess the data
X_preprocessed = preprocessor.transform(X)

# NO TARGET OR TRAIN TEST SPLIT
X.head()

,max_players,max_playtime,min_age,min_players,min_playtime,playing_time,category,mechanic,average_rating,users_rated,category_count,mechanic_count,has_expansion,len_description,description_sentiment
0,5,240,14,3,240,240,economic,area control / area influence,7.66508,4498,3,5,0,222,0.091012
1,4,30,12,3,30,30,card game,trick-taking,6.60815,478,2,1,0,154,0.055291
2,4,60,10,2,30,60,abstract strategy,area control / area influence,7.44119,12019,2,4,0,183,-0.056250
3,4,60,12,2,60,60,ancient,action point allowance system,6.60675,314,1,4,0,104,-0.007908
4,6,90,12,3,90,90,economic,hand management,7.35830,15195,1,3,0,191,0.168056


In [19]:
# define the number of nearest neighbors to consider
k = playlist_length

# Initialize the nearest neighbors model
model1 = NearestNeighbors(n_neighbors=k, metric="euclidean")

# Fit the model to the preprocessed data
model1.fit(X_preprocessed)

NearestNeighbors(metric='euclidean', n_neighbors=20)

In [20]:
# Extract the features for the specific track
game_features = df2.loc[df2.game_id == game_id, feature_cols]
game_features_preprocessed = preprocessor.transform(game_features)

# Find the nearest neighbors
distances, indices = model1.kneighbors(game_features_preprocessed)

# get the track names of the nearest neighbors
games = df2.iloc[indices[0]]
games["distance"] = distances[0]

# filter columns
cols = games.columns

games = games.loc[:, cols]
games = games.sort_values(by = "distance")
games.head(10)

,game_id,max_players,max_playtime,min_age,min_players,min_playtime,name,playing_time,category,mechanic,average_rating,users_rated,category_count,mechanic_count,has_expansion,len_description,description_sentiment,distance
12,13,4,120,10,3,60,Catan,120,negotiation,dice rolling,7.26569,67655,1,5,1,457,0.098867,0.000000
4702,30549,4,45,8,2,45,Pandemic,45,medical,action point allowance system,7.67234,62377,1,7,1,237,0.114088,3.768610
544,822,5,45,8,2,30,Carcassonne,45,city building,area control / area influence,7.43536,67056,3,2,1,204,-0.013889,4.135336
5041,36218,4,30,13,2,30,Dominion,30,card game,card drafting,7.69995,55930,2,3,1,289,0.130288,5.126014
5937,68448,7,30,10,2,30,7 Wonders,30,ancient,card drafting,7.83595,51688,4,5,1,252,-0.012574,6.535561
3052,9209,5,60,8,2,30,Ticket to Ride,60,trains,hand management,7.48301,48227,2,3,1,229,0.042562,7.300253
4739,31260,5,150,12,1,30,Agricola,150,animals,card drafting,8.05581,48261,3,4,1,343,0.082667,7.673573
1540,3076,5,150,12,2,90,Puerto Rico,150,city building,variable phase order,8.09157,47789,3,1,1,381,-0.017889,7.844523
5348,40692,5,80,8,2,40,Small World,80,fantasy,area control / area influence,7.35735,43144,3,4,1,250,0.084914,8.796502
1369,2651,6,120,12,2,120,Power Grid,120,economic,auction/bidding,7.94499,42036,2,2,1,215,0.051769,9.328175


In [ ]:
# try other distances

# define the number of nearest neighbors to consider
k = playlist_length

# Initialize the nearest neighbors model
model2 = NearestNeighbors(n_neighbors=k, metric="manhattan")

# Fit the model to the preprocessed data
model2.fit(X_preprocessed)

In [ ]:
# Extract the features for the specific track
track_features = df2.loc[df2.track_id == track_id, feature_cols]
track_features_preprocessed = preprocessor.transform(track_features)

# Find the nearest neighbors
distances, indices = model2.kneighbors(track_features_preprocessed)

# get the track names of the nearest neighbors
tracks = df2.iloc[indices[0]]
tracks["distance"] = distances[0]

# filter columns
cols = tracks.columns

tracks = tracks.loc[:, cols]
tracks = tracks.sort_values(by = "distance")
tracks.head(10)

In [ ]:
# try other distances

# define the number of nearest neighbors to consider
k = playlist_length

# Initialize the nearest neighbors model
model3 = NearestNeighbors(n_neighbors=k, metric="cosine")

# Fit the model to the preprocessed data
model3.fit(X_preprocessed)

In [ ]:
# Extract the features for the specific track
track_features = df2.loc[df2.track_id == track_id, feature_cols]
track_features_preprocessed = preprocessor.transform(track_features)

# Find the nearest neighbors
distances, indices = model3.kneighbors(track_features_preprocessed)

# get the track names of the nearest neighbors
tracks = df2.iloc[indices[0]]
tracks["distance"] = distances[0]

# filter columns
cols = tracks.columns

tracks = tracks.loc[:, cols]
tracks = tracks.sort_values(by = "distance")
tracks.head(10)